In [235]:
import requests
import json
import pandas as pd

In [236]:
endpoint = 'http://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.PP.KD'
params = {
    'format': 'json',
    'per_page': 16758
}
response = requests.get(endpoint, params)

data = response.json()

gdp_df = pd.json_normalize(data[1])

print(gdp_df.tail(10).to_string())



      countryiso3code  date  value unit obs_status  decimal       indicator.id                                      indicator.value country.id country.value
16748             ZWE  1969    NaN                        0  NY.GDP.PCAP.PP.KD  GDP per capita, PPP (constant 2017 international $)         ZW      Zimbabwe
16749             ZWE  1968    NaN                        0  NY.GDP.PCAP.PP.KD  GDP per capita, PPP (constant 2017 international $)         ZW      Zimbabwe
16750             ZWE  1967    NaN                        0  NY.GDP.PCAP.PP.KD  GDP per capita, PPP (constant 2017 international $)         ZW      Zimbabwe
16751             ZWE  1966    NaN                        0  NY.GDP.PCAP.PP.KD  GDP per capita, PPP (constant 2017 international $)         ZW      Zimbabwe
16752             ZWE  1965    NaN                        0  NY.GDP.PCAP.PP.KD  GDP per capita, PPP (constant 2017 international $)         ZW      Zimbabwe
16753             ZWE  1964    NaN                        

In [237]:
endpoint = 'http://api.worldbank.org/v2/country/all/indicator/SP.DYN.LE00.IN'
params = {
    'format': 'json',
    'per_page': 16758
}
response = requests.get(endpoint, params)

data = response.json()

le_df = pd.json_normalize(data[1])

print(le_df.head(10).to_string())

  countryiso3code  date      value unit obs_status  decimal    indicator.id                          indicator.value country.id                country.value
0             AFE  2022        NaN                        0  SP.DYN.LE00.IN  Life expectancy at birth, total (years)         ZH  Africa Eastern and Southern
1             AFE  2021  62.454590                        0  SP.DYN.LE00.IN  Life expectancy at birth, total (years)         ZH  Africa Eastern and Southern
2             AFE  2020  63.313860                        0  SP.DYN.LE00.IN  Life expectancy at birth, total (years)         ZH  Africa Eastern and Southern
3             AFE  2019  63.755678                        0  SP.DYN.LE00.IN  Life expectancy at birth, total (years)         ZH  Africa Eastern and Southern
4             AFE  2018  63.365863                        0  SP.DYN.LE00.IN  Life expectancy at birth, total (years)         ZH  Africa Eastern and Southern
5             AFE  2017  62.922390                        

In [238]:
columns_to_keep = ['country.value', 'date', 'value']

gdp_df = gdp_df[columns_to_keep]
le_df = le_df[columns_to_keep]

gdp_df = gdp_df.rename(columns={'value': 'gdp_per_cap'})
le_df = le_df.rename(columns={'value': 'life_exp'})
merged_df = gdp_df.merge(le_df, on=['country.value', 'date'], how='inner')

merged_df.dropna(inplace=True)
len(merged_df)

7365

In [239]:
endpoint = 'http://api.worldbank.org/v2/country'

params = {
    'format': 'json',
    'per_page': 16758
}
response = requests.get(endpoint, params)

data = response.json()

country_df = pd.json_normalize(data[1])

columns_to_keep = ['name', 'capitalCity', 'region.value', 'incomeLevel.value']
country_df = country_df[columns_to_keep]
country_df = country_df.rename(columns={'name': 'country.value'})


print(country_df.tail(10).to_string())

                                             country.value capitalCity                region.value    incomeLevel.value
287                                                Vietnam       Hanoi         East Asia & Pacific  Lower middle income
288                                                Vanuatu   Port-Vila         East Asia & Pacific  Lower middle income
289                                                  World                              Aggregates           Aggregates
290                                                  Samoa        Apia         East Asia & Pacific  Lower middle income
291                                                 Kosovo    Pristina       Europe & Central Asia  Upper middle income
292  Sub-Saharan Africa excluding South Africa and Nigeria                              Aggregates           Aggregates
293                                            Yemen, Rep.      Sana'a  Middle East & North Africa           Low income
294                                     

In [240]:
merged_df = merged_df.merge(country_df, on=['country.value'], how='inner')

merged_df = merged_df[merged_df['region.value'] != 'Aggregates']

merged_df.head(100)

,country.value,date,gdp_per_cap,life_exp,capitalCity,region.value,incomeLevel.value
1455,Afghanistan,2021,1517.016266,61.982,Kabul,South Asia,Low income
1456,Afghanistan,2020,1968.341002,62.575,Kabul,South Asia,Low income
1457,Afghanistan,2019,2079.921861,63.565,Kabul,South Asia,Low income
1458,Afghanistan,2018,2060.698973,63.081,Kabul,South Asia,Low income
1459,Afghanistan,2017,2096.093111,63.016,Kabul,South Asia,Low income
...,...,...,...,...,...,...,...
1550,Angola,2010,7689.820735,56.726,Luanda,Sub-Saharan Africa,Lower middle income
1551,Angola,2009,7646.144358,55.752,Luanda,Sub-Saharan Africa,Lower middle income
1552,Angola,2008,7866.184345,54.633,Luanda,Sub-Saharan Africa,Lower middle income
1553,Angola,2007,7340.388909,53.642,Luanda,Sub-Saharan Africa,Lower middle income
